# Project Template App - Run with Ngrok and Colab

In [1]:
!pip install flask-ngrok > /dev/null 2>&1
!pip install pyngrok > /dev/null 2>&1

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
with open('drive/MyDrive/project/static/ngrok_auth.txt') as nf:
    ngrok_auth = nf.read()

In [4]:
# 2DhgO5MjhmZSX4JI2hjppxKaiyA_29rMtfrrptS3nr3YkTXbd
!cat drive/MyDrive/project/static/ngrok_auth.txt 

2DhhM4zGyNGOWbPiOdKqnZOPYWi_2YmSF58Pc5289ZMLH5j3m

In [5]:
!ngrok authtoken $ngrok_auth

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [6]:
!pip install efficientnet_pytorch



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=9eb0e6caf2a2e45ba1f7f5bceda2f794cfe877f1e725c42051e1688f589b4473
  Stored in directory: /root/.cache/pip/wheels/0e/cc/b2/49e74588263573ff778da58cc99b9c6349b496636a7e165be6
Successfully built efficientnet-pytorch


### Web Server Code

In [7]:



from flask import Flask, render_template, request
from flask_ngrok import run_with_ngrok
import torchvision
from torchvision import transforms
import os

app = Flask(__name__, static_folder='/content/drive/MyDrive/project/static/upload', # 이곳에 유저가 업로드한 파일을 save
                      template_folder='/content/drive/MyDrive/project/templates')
run_with_ngrok(app)

@app.route('/')
def home():
    menu = {'home':1, 'menu':0}
    return render_template('index.html', menu=menu)

@app.route('/menu', methods=['GET','POST'])
def menu():
    menu = {'home':0, 'menu':1}
    if request.method == 'GET':
        languages = [
            #{'disp':'영어', 'val':'en'},
            #{'disp':'일어', 'val':'jp'},
            #{'disp':'중국어', 'val':'cn'},
            #{'disp':'프랑스어', 'val':'fr'},
            #{'disp':'스페인어', 'val':'es'}
        ]
        
        return render_template('menu.html', menu=menu,
                                options=languages)   # 서버에서 클라이언트로 정보 전달
    else:
        # 사용자가 입력한 정보를 서버가 읽음
        # index = request.form['index']
        # lang = request.form['lang']
        # lyrics = request.form['lyrics']
        #print(lang, '\n', index, '\n', lyrics, sep='')
        # 사용자가 입력한 파일을 읽어서 upload 디렉토리에 저장
        f_image = request.files['image']
        fname = f_image.filename                # 사용자가 입력한 파일 이름
        filename = os.path.join(app.static_folder, 'upload/') + fname  # 유저가 업로드한 사진이 저장되는 공간과 파일이름 
                                        # static_folder/upload 가 경로 파일네임 : fname 유저가 업로드한 파일의 이름 
        f_image.save(filename) #파일세이브
       

##########################################################################################################################
##########################################################################################################################
##########################################################################################################################
##########################################################################################################################
##########################################################################################################################
##########################################################################################################################
##########################################################################################################################
##########################################################################################################################
##########################################################################################################################
##########################################################################################################################
##########################################################################################################################

     
        PATH = '/content/drive/MyDrive/Colab Notebooks/scalp_weights/'+'aram_model1.pt'  # 모델1
        # PATH2 = '/content/drive/MyDrive/Colab Notebooks/scalp_weights/'+'president_aram_model1.pt' # 모델1 파라미터
        # CUDA 를 활용한 GPU 가속 여부에 따라, 장치를 할당 할 수 있도록 변수로 선언
        import torch
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        # 모델1 불러오기
        model1 = torch.load(PATH, map_location=device)
        # 모델1 파라미터 불러오기
        # model1_p = torch.load(PATH2, map_location=device)
        ## test 이미지파일 전처리, 텐서화
        
        # 전처리-트랜스폼 규칙 선언 # model1_train 코드의 validation set 의 트랜스폼 규칙과 동일하게 함
        transforms_test = transforms.Compose([
                                                transforms.Resize([int(600), int(600)], interpolation=transforms.InterpolationMode.BICUBIC),
                                                transforms.ToTensor(),
                                                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                                            ])
        # root 경로 폴더 속 jpg를 전처리, 텐서화 (rood 속에 폴더를 하나 더 만들어서 jpg를 묶어야 함)
        testset = torchvision.datasets.ImageFolder(root = '/content/drive/MyDrive/project/static/upload' ,
                            transform = transforms_test)
        ## 확인
        testset.__getitem__(0)[0].shape  # (0) 테스트셋의 0번째 item
        # rgb three channel 이니까 3
        # 높이 넓이 600 600 리사이즈
        # 전처리 정상 완료 확인
        # DataLoader를 통해 네트워크에 올리기
        from torch.utils.data import Dataset,DataLoader 
        testloader = DataLoader(testset, batch_size=1, shuffle=False, num_workers=2)
        # dataiter = iter(testloader)
        # images = dataiter.next()
        # images[0].shape # [1, 3, 600, 600] # 처음1은 배치사이즈와 동일
        ## 아웃풋, 로스, 프레딕, 아큐러시
        output_list = []
        model1.eval() # 평가모드로 전환 # 평가모드와 학습모드의 layer 구성이 다르다
        correct = 0
        # 로스 연산
        # import torch.nn.functional as F   # F : 테스트_로스 연산 함수
        # test_loss = 0
        from tqdm import tqdm # 진행률 표시를 위한

        if __name__ == '__main__':
            with torch.no_grad(): # 평가할 땐  gradient를 backpropagation 하지 않기 때문에 no grad로 gradient 계산을 막아서 연산 속도를 높인다
                    for data, target in tqdm(testloader):                                   
                        data, target  = data.to(device), target.to(device) 
                        output = model1(data)   # model1에 데이터를 넣어서 아웃풋 > [a,b,c,d] 각 0,1,2,3 의 확률값 리턴 가장 큰 것이 pred
                        output_list.append(output)

        m1output = output_list[-1]
        m1predict = output_list[-1].argmax(dim=1, keepdim=True)


        ## 모델2

        PATH = '/content/drive/MyDrive/Colab Notebooks/scalp_weights/'+'aram_model2.pt'  # 모델1
        # PATH2 = '/content/drive/MyDrive/Colab Notebooks/scalp_weights/'+'president_aram_model1.pt' # 모델1 파라미터
        # CUDA 를 활용한 GPU 가속 여부에 따라, 장치를 할당 할 수 있도록 변수로 선언
        # import torch
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        # 모델1 불러오기
        # !pip install efficientnet_pytorch
        model2 = torch.load(PATH, map_location=device)
        # 모델1 파라미터 불러오기
        # model1_p = torch.load(PATH2, map_location=device)
        ## test 이미지파일 전처리, 텐서화
        # import torchvision
        # from torchvision import transforms
        # 전처리-트랜스폼 규칙 선언 # model1_train 코드의 validation set 의 트랜스폼 규칙과 동일하게 함
        transforms_test = transforms.Compose([
                                                transforms.Resize([int(600), int(600)], interpolation=transforms.InterpolationMode.BICUBIC),
                                                transforms.ToTensor(),
                                                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                                            ])
        # root 경로 폴더 속 jpg를 전처리, 텐서화 (rood 속에 폴더를 하나 더 만들어서 jpg를 묶어야 함)
        testset = torchvision.datasets.ImageFolder(root =  '/content/drive/MyDrive/project/static/upload', 
                            transform = transforms_test)
        ## 확인
        testset.__getitem__(0)[0].shape  # (0) 테스트셋의 0번째 item
        # rgb three channel 이니까 3
        # 높이 넓이 600 600 리사이즈
        # 전처리 정상 완료 확인
        # DataLoader를 통해 네트워크에 올리기
        # from torch.utils.data import Dataset,DataLoader 
        testloader = DataLoader(testset, batch_size=1, shuffle=False, num_workers=2)
        # dataiter = iter(testloader)
        # images = dataiter.next()
        # images[0].shape # [1, 3, 600, 600] # 처음1은 배치사이즈와 동일
        ## 아웃풋, 로스, 프레딕, 아큐러시
        output_list = []
        model2.eval() # 평가모드로 전환 # 평가모드와 학습모드의 layer 구성이 다르다
        correct = 0
        # 로스 연산
        # import torch.nn.functional as F   # F : 테스트_로스 연산 함수
        # test_loss = 0
        # from tqdm import tqdm # 진행률 표시를 위한

        if __name__ == '__main__':
            with torch.no_grad(): # 평가할 땐  gradient를 backpropagation 하지 않기 때문에 no grad로 gradient 계산을 막아서 연산 속도를 높인다
                    for data, target in tqdm(testloader):                                   
                        data, target  = data.to(device), target.to(device) 
                        output = model2(data)   # model1에 데이터를 넣어서 아웃풋 > [a,b,c,d] 각 0,1,2,3 의 확률값 리턴 가장 큰 것이 pred
                        output_list.append(output);
                        # test_loss += F.nll_loss(output, target, reduction = 'sum').item()  # test_loss변수에 각 로스를 축적
                        # pred = output.argmax(dim=1, keepdim=True) # argmax : 리스트에서 최댓값의 인덱스를 뽑아줌 > y값아웃풋인덱
                        # correct += pred.eq(target.view_as(pred)).sum().item() # accuracy 측정을 위한 변수 # 각 예측이 맞았는지 틀렸는지 correct변수에 축적 맞을 때마다 +1  # # view_as() 함수는 target 텐서를 view_as() 함수 안에 들어가는 인수(pred)의 모양대로 다시 정렬한다.
                    #  view_as() 함수는 target 텐서를 view_as() 함수 안에 들어가는 인수(pred)의 모양대로 다시 정렬한다.
                    #  pred.eq(data) : pred와 data가 일치하는지 검사
        # test_loss /= len(testloader.dataset)  # 로스축적된 로스를 데이터 수(경로안jpg수)로 나누기
        # 아큐러시 출력 ( :.4f 소수점반올림 )
        # print('\nTest set Accuracy: {}/{} ({:.4f}%)\n'.format(correct, len(testloader.dataset), 100. * correct / len(testloader.dataset)))  # 축적된 예측값을 데이터 개수로 나누기 *100 > 확률%값
        # 로스, 아큐러시 출력
        # print('\nTest set: Average Loss: {:.4f}, Accuracy: {}/{} ({:.4f}%)\n'.format(test_loss, correct, len(testloader.dataset), 100. * correct / len(testloader.dataset)))

        # 업로드 이미지에 대한 output predict 값
        # 업로드 폴더로 실행 > 아웃풋리스트에서 최근 업로드 파일을 -1인덱스로 잡은 후 아웃풋 리스트에서 -1 인덱스로 최근 업로드 파일을 잡고 argmax로 아웃풋 0 1 2 3 중 최댓값의 인덱스를 리턴

        #  0:양호, 1:경증, 2:중등도, 3:중증
        m2output = output_list[-1]
        m2predict = output_list[-1].argmax(dim=1, keepdim=True)
        #print()
        #print()
        #print('model2 피지과다 output :', m2output) # tensor([[-7.3876,  1.1129,  2.3978, -0.7094]], device='cuda:0') 로 각 아웃풋 4개에 대한 수치를 알려줌
        #print('model2 피지과다 predict :',m2predict)   # 위 값 중 최댓값의 인덱스를 알려줌 > 즉 predict 값

        # 모델3 upload file predict
        
        # 모델 2~6 으로 코드 변경시 변경해야 할 것
        # PATH = '/content/drive/MyDrive/Colab Notebooks/scalp_weights/'+'aram_model1.pt'  # aram_model1.pt 여기서 1 > 2,3,4,5,6
        # model1 = torch.load(PATH, map_location=device)   # mode1 > 2,3,4,5,6
        # testset = torchvision.datasets.ImageFolder(root = '/content/drive/MyDrive/Colab Notebooks/train_data/model1/test',    # 여기서 1 > 2,3,4,5,6 (폴더는 학습에서 잡혀있음)
        # model1.eval()
        # output = model1(data) 

        # 구글드라이브 마운트
        # from google.colab import drive
        # drive.mount('/content/drive')  
        # 모델.pt파일 경로
        PATH = '/content/drive/MyDrive/Colab Notebooks/scalp_weights/'+'aram_model3.pt'  # 모델1
        # PATH2 = '/content/drive/MyDrive/Colab Notebooks/scalp_weights/'+'president_aram_model1.pt' # 모델1 파라미터
        # CUDA 를 활용한 GPU 가속 여부에 따라, 장치를 할당 할 수 있도록 변수로 선언
        # import torch
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        # 모델1 불러오기
        # !pip install efficientnet_pytorch
        model3 = torch.load(PATH, map_location=device)
        # 모델1 파라미터 불러오기
        # model1_p = torch.load(PATH2, map_location=device)
        ## test 이미지파일 전처리, 텐서화
        # import torchvision
        # from torchvision import transforms
        # 전처리-트랜스폼 규칙 선언 # model1_train 코드의 validation set 의 트랜스폼 규칙과 동일하게 함
        transforms_test = transforms.Compose([
                                                transforms.Resize([int(600), int(600)], interpolation=transforms.InterpolationMode.BICUBIC),
                                                transforms.ToTensor(),
                                                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                                            ])
        # root 경로 폴더 속 jpg를 전처리, 텐서화 (rood 속에 폴더를 하나 더 만들어서 jpg를 묶어야 함)
        testset = torchvision.datasets.ImageFolder(root =  '/content/drive/MyDrive/project/static/upload',  
                            transform = transforms_test)
        ## 확인
        testset.__getitem__(0)[0].shape  # (0) 테스트셋의 0번째 item
        # rgb three channel 이니까 3
        # 높이 넓이 600 600 리사이즈
        # 전처리 정상 완료 확인
        # DataLoader를 통해 네트워크에 올리기
        # from torch.utils.data import Dataset,DataLoader 
        testloader = DataLoader(testset, batch_size=1, shuffle=False, num_workers=2)
        # dataiter = iter(testloader)
        # images = dataiter.next()
        # images[0].shape # [1, 3, 600, 600] # 처음1은 배치사이즈와 동일
        ## 아웃풋, 로스, 프레딕, 아큐러시
        output_list = []
        model3.eval() # 평가모드로 전환 # 평가모드와 학습모드의 layer 구성이 다르다
        correct = 0
        # 로스 연산
        # import torch.nn.functional as F   # F : 테스트_로스 연산 함수
        # test_loss = 0
        # from tqdm import tqdm # 진행률 표시를 위한

        if __name__ == '__main__':
            with torch.no_grad(): # 평가할 땐  gradient를 backpropagation 하지 않기 때문에 no grad로 gradient 계산을 막아서 연산 속도를 높인다
                    for data, target in tqdm(testloader):                                   
                        data, target  = data.to(device), target.to(device) 
                        output = model3(data)   # model1에 데이터를 넣어서 아웃풋 > [a,b,c,d] 각 0,1,2,3 의 확률값 리턴 가장 큰 것이 pred
                        output_list.append(output);
                        # test_loss += F.nll_loss(output, target, reduction = 'sum').item()  # test_loss변수에 각 로스를 축적
                        # pred = output.argmax(dim=1, keepdim=True) # argmax : 리스트에서 최댓값의 인덱스를 뽑아줌 > y값아웃풋인덱
                        # correct += pred.eq(target.view_as(pred)).sum().item() # accuracy 측정을 위한 변수 # 각 예측이 맞았는지 틀렸는지 correct변수에 축적 맞을 때마다 +1  # # view_as() 함수는 target 텐서를 view_as() 함수 안에 들어가는 인수(pred)의 모양대로 다시 정렬한다.
                    #  view_as() 함수는 target 텐서를 view_as() 함수 안에 들어가는 인수(pred)의 모양대로 다시 정렬한다.
                    #  pred.eq(data) : pred와 data가 일치하는지 검사
        # test_loss /= len(testloader.dataset)  # 로스축적된 로스를 데이터 수(경로안jpg수)로 나누기
        # 아큐러시 출력 ( :.4f 소수점반올림 )
        # print('\nTest set Accuracy: {}/{} ({:.4f}%)\n'.format(correct, len(testloader.dataset), 100. * correct / len(testloader.dataset)))  # 축적된 예측값을 데이터 개수로 나누기 *100 > 확률%값
        # 로스, 아큐러시 출력
        # print('\nTest set: Average Loss: {:.4f}, Accuracy: {}/{} ({:.4f}%)\n'.format(test_loss, correct, len(testloader.dataset), 100. * correct / len(testloader.dataset)))

        # 업로드 이미지에 대한 output predict 값
        # 업로드 폴더로 실행 > 아웃풋리스트에서 최근 업로드 파일을 -1인덱스로 잡은 후 아웃풋 리스트에서 -1 인덱스로 최근 업로드 파일을 잡고 argmax로 아웃풋 0 1 2 3 중 최댓값의 인덱스를 리턴

        #  0:양호, 1:경증, 2:중등도, 3:중증
        m3output = output_list[-1]
        m3predict = output_list[-1].argmax(dim=1, keepdim=True)
        #print()
        #print()
        #print('model3 모낭사이홍반 output :', m3output) # tensor([[-7.3876,  1.1129,  2.3978, -0.7094]], device='cuda:0') 로 각 아웃풋 4개에 대한 수치를 알려줌
        #print('model3 모낭사이홍반 predict :',m3predict)   # 위 값 중 최댓값의 인덱스를 알려줌 > 즉 predict 값

        # 모델4 upload file predict
        
        # 모델 2~6 으로 코드 변경시 변경해야 할 것
        # PATH = '/content/drive/MyDrive/Colab Notebooks/scalp_weights/'+'aram_model1.pt'  # aram_model1.pt 여기서 1 > 2,3,4,5,6
        # model1 = torch.load(PATH, map_location=device)   # mode1 > 2,3,4,5,6
        # testset = torchvision.datasets.ImageFolder(root = '/content/drive/MyDrive/Colab Notebooks/train_data/model1/test',    # 여기서 1 > 2,3,4,5,6 (폴더는 학습에서 잡혀있음)
        # model1.eval()
        # output = model1(data) 

        # 구글드라이브 마운트
        # from google.colab import drive
        # drive.mount('/content/drive')  
        # 모델.pt파일 경로
        PATH = '/content/drive/MyDrive/Colab Notebooks/scalp_weights/'+'aram_model4.pt'  # 모델1
        # PATH2 = '/content/drive/MyDrive/Colab Notebooks/scalp_weights/'+'president_aram_model1.pt' # 모델1 파라미터
        # CUDA 를 활용한 GPU 가속 여부에 따라, 장치를 할당 할 수 있도록 변수로 선언
        # import torch
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        # 모델1 불러오기
        # !pip install efficientnet_pytorch
        model4 = torch.load(PATH, map_location=device)
        # 모델1 파라미터 불러오기
        # model1_p = torch.load(PATH2, map_location=device)
        ## test 이미지파일 전처리, 텐서화
        # import torchvision
        # from torchvision import transforms
        # 전처리-트랜스폼 규칙 선언 # model1_train 코드의 validation set 의 트랜스폼 규칙과 동일하게 함
        transforms_test = transforms.Compose([
                                                transforms.Resize([int(600), int(600)], interpolation=transforms.InterpolationMode.BICUBIC),
                                                transforms.ToTensor(),
                                                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                                            ])
        # root 경로 폴더 속 jpg를 전처리, 텐서화 (rood 속에 폴더를 하나 더 만들어서 jpg를 묶어야 함)
        testset = torchvision.datasets.ImageFolder(root =  '/content/drive/MyDrive/project/static/upload',  
                            transform = transforms_test)
        ## 확인
        testset.__getitem__(0)[0].shape  # (0) 테스트셋의 0번째 item
        # rgb three channel 이니까 3
        # 높이 넓이 600 600 리사이즈
        # 전처리 정상 완료 확인
        # DataLoader를 통해 네트워크에 올리기
        # from torch.utils.data import Dataset,DataLoader 
        testloader = DataLoader(testset, batch_size=1, shuffle=False, num_workers=2)
        # dataiter = iter(testloader)
        # images = dataiter.next()
        # images[0].shape # [1, 3, 600, 600] # 처음1은 배치사이즈와 동일
        ## 아웃풋, 로스, 프레딕, 아큐러시
        output_list = []
        model4.eval() # 평가모드로 전환 # 평가모드와 학습모드의 layer 구성이 다르다
        correct = 0
        # 로스 연산
        # import torch.nn.functional as F   # F : 테스트_로스 연산 함수
        # test_loss = 0
        # from tqdm import tqdm # 진행률 표시를 위한

        if __name__ == '__main__':
            with torch.no_grad(): # 평가할 땐  gradient를 backpropagation 하지 않기 때문에 no grad로 gradient 계산을 막아서 연산 속도를 높인다
                    for data, target in tqdm(testloader):                                   
                        data, target  = data.to(device), target.to(device) 
                        output = model4(data)   # model1에 데이터를 넣어서 아웃풋 > [a,b,c,d] 각 0,1,2,3 의 확률값 리턴 가장 큰 것이 pred
                        output_list.append(output);
                        # test_loss += F.nll_loss(output, target, reduction = 'sum').item()  # test_loss변수에 각 로스를 축적
                        # pred = output.argmax(dim=1, keepdim=True) # argmax : 리스트에서 최댓값의 인덱스를 뽑아줌 > y값아웃풋인덱
                        # correct += pred.eq(target.view_as(pred)).sum().item() # accuracy 측정을 위한 변수 # 각 예측이 맞았는지 틀렸는지 correct변수에 축적 맞을 때마다 +1  # # view_as() 함수는 target 텐서를 view_as() 함수 안에 들어가는 인수(pred)의 모양대로 다시 정렬한다.
                    #  view_as() 함수는 target 텐서를 view_as() 함수 안에 들어가는 인수(pred)의 모양대로 다시 정렬한다.
                    #  pred.eq(data) : pred와 data가 일치하는지 검사
        # test_loss /= len(testloader.dataset)  # 로스축적된 로스를 데이터 수(경로안jpg수)로 나누기
        # 아큐러시 출력 ( :.4f 소수점반올림 )
        # print('\nTest set Accuracy: {}/{} ({:.4f}%)\n'.format(correct, len(testloader.dataset), 100. * correct / len(testloader.dataset)))  # 축적된 예측값을 데이터 개수로 나누기 *100 > 확률%값
        # 로스, 아큐러시 출력
        # print('\nTest set: Average Loss: {:.4f}, Accuracy: {}/{} ({:.4f}%)\n'.format(test_loss, correct, len(testloader.dataset), 100. * correct / len(testloader.dataset)))

        # 업로드 이미지에 대한 output predict 값
        # 업로드 폴더로 실행 > 아웃풋리스트에서 최근 업로드 파일을 -1인덱스로 잡은 후 아웃풋 리스트에서 -1 인덱스로 최근 업로드 파일을 잡고 argmax로 아웃풋 0 1 2 3 중 최댓값의 인덱스를 리턴

        #  0:양호, 1:경증, 2:중등도, 3:중증
        m4output = output_list[-1]
        m4predict = output_list[-1].argmax(dim=1, keepdim=True)
        #print()
        #print()
        #print('model4 모낭홍반농포 output :', m4output) # tensor([[-7.3876,  1.1129,  2.3978, -0.7094]], device='cuda:0') 로 각 아웃풋 4개에 대한 수치를 알려줌
        #print('model4 모낭홍반농포 predict :',m4predict)   # 위 값 중 최댓값의 인덱스를 알려줌 > 즉 predict 값

        # 모델5 upload file predict
        
        # 모델 2~6 으로 코드 변경시 변경해야 할 것
        # PATH = '/content/drive/MyDrive/Colab Notebooks/scalp_weights/'+'aram_model1.pt'  # aram_model1.pt 여기서 1 > 2,3,4,5,6
        # model1 = torch.load(PATH, map_location=device)   # mode1 > 2,3,4,5,6
        # testset = torchvision.datasets.ImageFolder(root = '/content/drive/MyDrive/Colab Notebooks/train_data/model1/test',    # 여기서 1 > 2,3,4,5,6 (폴더는 학습에서 잡혀있음)
        # model1.eval()
        # output = model1(data) 

        # 구글드라이브 마운트
        # from google.colab import drive
        # drive.mount('/content/drive')  
        # 모델.pt파일 경로
        PATH = '/content/drive/MyDrive/Colab Notebooks/scalp_weights/'+'aram_model5.pt'  # 모델1
        # PATH2 = '/content/drive/MyDrive/Colab Notebooks/scalp_weights/'+'president_aram_model1.pt' # 모델1 파라미터
        # CUDA 를 활용한 GPU 가속 여부에 따라, 장치를 할당 할 수 있도록 변수로 선언
        # import torch
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        # 모델1 불러오기
        # !pip install efficientnet_pytorch
        model5 = torch.load(PATH, map_location=device)
        # 모델1 파라미터 불러오기
        # model1_p = torch.load(PATH2, map_location=device)
        ## test 이미지파일 전처리, 텐서화
        # import torchvision
        # from torchvision import transforms
        # 전처리-트랜스폼 규칙 선언 # model1_train 코드의 validation set 의 트랜스폼 규칙과 동일하게 함
        transforms_test = transforms.Compose([
                                                transforms.Resize([int(600), int(600)], interpolation=transforms.InterpolationMode.BICUBIC),
                                                transforms.ToTensor(),
                                                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                                            ])
        # root 경로 폴더 속 jpg를 전처리, 텐서화 (rood 속에 폴더를 하나 더 만들어서 jpg를 묶어야 함)
        testset = torchvision.datasets.ImageFolder(root = '/content/drive/MyDrive/project/static/upload',  
                            transform = transforms_test)
        ## 확인
        testset.__getitem__(0)[0].shape  # (0) 테스트셋의 0번째 item
        # rgb three channel 이니까 3
        # 높이 넓이 600 600 리사이즈
        # 전처리 정상 완료 확인
        # DataLoader를 통해 네트워크에 올리기
        # from torch.utils.data import Dataset,DataLoader 
        testloader = DataLoader(testset, batch_size=1, shuffle=False, num_workers=2)
        # dataiter = iter(testloader)
        # images = dataiter.next()
        # images[0].shape # [1, 3, 600, 600] # 처음1은 배치사이즈와 동일
        ## 아웃풋, 로스, 프레딕, 아큐러시
        output_list = []
        model5.eval() # 평가모드로 전환 # 평가모드와 학습모드의 layer 구성이 다르다
        correct = 0
        # 로스 연산
        # import torch.nn.functional as F   # F : 테스트_로스 연산 함수
        # test_loss = 0
        # from tqdm import tqdm # 진행률 표시를 위한

        if __name__ == '__main__':
            with torch.no_grad(): # 평가할 땐  gradient를 backpropagation 하지 않기 때문에 no grad로 gradient 계산을 막아서 연산 속도를 높인다
                    for data, target in tqdm(testloader):                                   
                        data, target  = data.to(device), target.to(device) 
                        output = model5(data)   # model1에 데이터를 넣어서 아웃풋 > [a,b,c,d] 각 0,1,2,3 의 확률값 리턴 가장 큰 것이 pred
                        output_list.append(output);
                        # test_loss += F.nll_loss(output, target, reduction = 'sum').item()  # test_loss변수에 각 로스를 축적
                        # pred = output.argmax(dim=1, keepdim=True) # argmax : 리스트에서 최댓값의 인덱스를 뽑아줌 > y값아웃풋인덱
                        # correct += pred.eq(target.view_as(pred)).sum().item() # accuracy 측정을 위한 변수 # 각 예측이 맞았는지 틀렸는지 correct변수에 축적 맞을 때마다 +1  # # view_as() 함수는 target 텐서를 view_as() 함수 안에 들어가는 인수(pred)의 모양대로 다시 정렬한다.
                    #  view_as() 함수는 target 텐서를 view_as() 함수 안에 들어가는 인수(pred)의 모양대로 다시 정렬한다.
                    #  pred.eq(data) : pred와 data가 일치하는지 검사
        # test_loss /= len(testloader.dataset)  # 로스축적된 로스를 데이터 수(경로안jpg수)로 나누기
        # 아큐러시 출력 ( :.4f 소수점반올림 )
        # print('\nTest set Accuracy: {}/{} ({:.4f}%)\n'.format(correct, len(testloader.dataset), 100. * correct / len(testloader.dataset)))  # 축적된 예측값을 데이터 개수로 나누기 *100 > 확률%값
        # 로스, 아큐러시 출력
        # print('\nTest set: Average Loss: {:.4f}, Accuracy: {}/{} ({:.4f}%)\n'.format(test_loss, correct, len(testloader.dataset), 100. * correct / len(testloader.dataset)))

        # 업로드 이미지에 대한 output predict 값
        # 업로드 폴더로 실행 > 아웃풋리스트에서 최근 업로드 파일을 -1인덱스로 잡은 후 아웃풋 리스트에서 -1 인덱스로 최근 업로드 파일을 잡고 argmax로 아웃풋 0 1 2 3 중 최댓값의 인덱스를 리턴

        #  0:양호, 1:경증, 2:중등도, 3:중증
        m5output = output_list[-1]
        m5predict = output_list[-1].argmax(dim=1, keepdim=True)
        #print()
        #print()
        #print('model5 비듬 output :', m5output) # tensor([[-7.3876,  1.1129,  2.3978, -0.7094]], device='cuda:0') 로 각 아웃풋 4개에 대한 수치를 알려줌
        #print('model5 비듬 predict :',m5predict)   # 위 값 중 최댓값의 인덱스를 알려줌 > 즉 predict 값

        # 모델6 upload file predict
        
        # 모델 2~6 으로 코드 변경시 변경해야 할 것
        # PATH = '/content/drive/MyDrive/Colab Notebooks/scalp_weights/'+'aram_model1.pt'  # aram_model1.pt 여기서 1 > 2,3,4,5,6
        # model1 = torch.load(PATH, map_location=device)   # mode1 > 2,3,4,5,6
        # testset = torchvision.datasets.ImageFolder(root = '/content/drive/MyDrive/Colab Notebooks/train_data/model1/test',    # 여기서 1 > 2,3,4,5,6 (폴더는 학습에서 잡혀있음)
        # model1.eval()
        # output = model1(data) 

        # 구글드라이브 마운트
        # from google.colab import drive
        # drive.mount('/content/drive')  
        # 모델.pt파일 경로
        PATH = '/content/drive/MyDrive/Colab Notebooks/scalp_weights/'+'aram_model6.pt'  # 모델1
        # PATH2 = '/content/drive/MyDrive/Colab Notebooks/scalp_weights/'+'president_aram_model1.pt' # 모델1 파라미터
        # CUDA 를 활용한 GPU 가속 여부에 따라, 장치를 할당 할 수 있도록 변수로 선언
        # import torch
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        # 모델1 불러오기
        # !pip install efficientnet_pytorch
        model6 = torch.load(PATH, map_location=device)
        # 모델1 파라미터 불러오기
        # model1_p = torch.load(PATH2, map_location=device)
        ## test 이미지파일 전처리, 텐서화
        # import torchvision
        # from torchvision import transforms
        # 전처리-트랜스폼 규칙 선언 # model1_train 코드의 validation set 의 트랜스폼 규칙과 동일하게 함
        transforms_test = transforms.Compose([
                                                transforms.Resize([int(600), int(600)], interpolation=transforms.InterpolationMode.BICUBIC),
                                                transforms.ToTensor(),
                                                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                                            ])
        # root 경로 폴더 속 jpg를 전처리, 텐서화 (rood 속에 폴더를 하나 더 만들어서 jpg를 묶어야 함)
        testset = torchvision.datasets.ImageFolder(root = '/content/drive/MyDrive/project/static/upload',  
                            transform = transforms_test)
        ## 확인
        testset.__getitem__(0)[0].shape  # (0) 테스트셋의 0번째 item
        # rgb three channel 이니까 3
        # 높이 넓이 600 600 리사이즈
        # 전처리 정상 완료 확인
        # DataLoader를 통해 네트워크에 올리기
        # from torch.utils.data import Dataset,DataLoader 
        testloader = DataLoader(testset, batch_size=1, shuffle=False, num_workers=2)
        # dataiter = iter(testloader)
        # images = dataiter.next()
        # images[0].shape # [1, 3, 600, 600] # 처음1은 배치사이즈와 동일
        ## 아웃풋, 로스, 프레딕, 아큐러시
        output_list = []
        model6.eval() # 평가모드로 전환 # 평가모드와 학습모드의 layer 구성이 다르다
        correct = 0
        # 로스 연산
        # import torch.nn.functional as F   # F : 테스트_로스 연산 함수
        # test_loss = 0
        # from tqdm import tqdm # 진행률 표시를 위한

        if __name__ == '__main__':
            with torch.no_grad(): # 평가할 땐  gradient를 backpropagation 하지 않기 때문에 no grad로 gradient 계산을 막아서 연산 속도를 높인다
                    for data, target in tqdm(testloader):                                   
                        data, target  = data.to(device), target.to(device) 
                        output = model6(data)   # model1에 데이터를 넣어서 아웃풋 > [a,b,c,d] 각 0,1,2,3 의 확률값 리턴 가장 큰 것이 pred
                        output_list.append(output);
                        # test_loss += F.nll_loss(output, target, reduction = 'sum').item()  # test_loss변수에 각 로스를 축적
                        # pred = output.argmax(dim=1, keepdim=True) # argmax : 리스트에서 최댓값의 인덱스를 뽑아줌 > y값아웃풋인덱
                        # correct += pred.eq(target.view_as(pred)).sum().item() # accuracy 측정을 위한 변수 # 각 예측이 맞았는지 틀렸는지 correct변수에 축적 맞을 때마다 +1  # # view_as() 함수는 target 텐서를 view_as() 함수 안에 들어가는 인수(pred)의 모양대로 다시 정렬한다.
                    #  view_as() 함수는 target 텐서를 view_as() 함수 안에 들어가는 인수(pred)의 모양대로 다시 정렬한다.
                    #  pred.eq(data) : pred와 data가 일치하는지 검사
        # test_loss /= len(testloader.dataset)  # 로스축적된 로스를 데이터 수(경로안jpg수)로 나누기
        # 아큐러시 출력 ( :.4f 소수점반올림 )
        # print('\nTest set Accuracy: {}/{} ({:.4f}%)\n'.format(correct, len(testloader.dataset), 100. * correct / len(testloader.dataset)))  # 축적된 예측값을 데이터 개수로 나누기 *100 > 확률%값
        # 로스, 아큐러시 출력
        # print('\nTest set: Average Loss: {:.4f}, Accuracy: {}/{} ({:.4f}%)\n'.format(test_loss, correct, len(testloader.dataset), 100. * correct / len(testloader.dataset)))

        # 업로드 이미지에 대한 output predict 값
        # 업로드 폴더로 실행 > 아웃풋리스트에서 최근 업로드 파일을 -1인덱스로 잡은 후 아웃풋 리스트에서 -1 인덱스로 최근 업로드 파일을 잡고 argmax로 아웃풋 0 1 2 3 중 최댓값의 인덱스를 리턴

        #  0:양호, 1:경증, 2:중등도, 3:중증
        m6output = output_list[-1]
        m6predict = output_list[-1].argmax(dim=1, keepdim=True)
        #print()
        #print()
        #print('model6 탈모 output :', m6output) # tensor([[-7.3876,  1.1129,  2.3978, -0.7094]], device='cuda:0') 로 각 아웃풋 4개에 대한 수치를 알려줌
        #print('model6 탈모 predict :',m6predict)   # 위 값 중 최댓값의 인덱스를 알려줌 > 즉 predict 값


        #print()
        #print()

        # 진단

        m1p = m1predict[0][0].tolist()
        m2p = m2predict[0][0].tolist() 
        m3p = m3predict[0][0].tolist()  
        m4p = m4predict[0][0].tolist() 
        m5p = m5predict[0][0].tolist() 
        m6p = m6predict[0][0].tolist()

        d_list = []

        if m1p == 0 and m2p == 0 and m3p == 0 and m4p == 0 and m5p == 0 and m6p == 0 :
            d1 = '정상입니다.'
            d_list.append(d1)
        elif m1p != 0 and m2p == 0 and m3p == 0 and m4p == 0 and m5p == 0 and m6p == 0 :
            d2 = '건성 두피입니다.'
            d_list.append(d2)
        elif m1p == 0 and m2p != 0 and m3p == 0 and m4p == 0 and m5p == 0 and m6p == 0 :
            d3 = '지성 두피입니다.'
            d_list.append(d3)
        elif m2p == 0 and m3p != 0 and m4p == 0 and m5p == 0 and m6p == 0 :
            d4 = '민감성 두피입니다.'
            d_list.append(d4)
        elif m2p != 0 and m3p != 0 and m4p == 0 and m6p == 0 :
            d5 = '지루성 두피입니다.'
            d_list.append(d5)
        elif m3p == 0 and m4p != 0 and m6p == 0 :
            d6 = '염증성 두피입니다.'
            d_list.append(d6)
        elif m3p == 0 and m4p == 0 and m5p != 0 and m6p == 0 :
            d7 = '비듬성 두피입니다.'
            d_list.append(d7)
        elif m1p == 0 and m2p != 0 and m3p == 0 and m4p == 0 and m5p == 0 and m6p != 0 :
            d8 = '탈모입니다.'
            d_list.append(d8)
        else:
            d9 = '복합성 두피입니다.'
            d_list.append(d9)

        #print('0:양호, 1:경증, 2:중등도, 3:중증',end='\n\n')
        #print('미세각질 :',m1p )
        #print('피지과다 :',m2p ) 
        #print('모낭사이홍반 :',m3p )  
        #print('모낭홍반농포 :',m4p ) 
        #print('비듬 :',m5p ) 
        #print('탈모 :',m6p ) 














##########################################################################################################################
##########################################################################################################################
##########################################################################################################################
##########################################################################################################################
##########################################################################################################################
##########################################################################################################################
##########################################################################################################################
##########################################################################################################################
##########################################################################################################################
##########################################################################################################################
##########################################################################################################################
 


        # 모델 실행후 결과를 돌려줌
        
        final = d_list[0]
        result = {'미세각질':m1p, '피지과다':m2p,'모낭사이홍반':m3p,'모낭홍반농포':m4p,'비듬':m5p,'탈모':m6p}
        final2 = '0:양호, 1:경증, 2:중등도, 3:중증'
        #result =  d_list, f'''미세각질 : {m1p}, 피지과다 : {m2p}, 모낭사이홍반 : {m3p}, 모낭홍반농포 : {m4p}, 비듬 : {m5p}, 탈모 : {m6p}, (0:양호, 1:경증, 2:중등도, 3:중증)'''
        mtime = int(os.stat(filename).st_mtime) # 업로드한 시간값불러오기 > 큐변경 > 화면갱신 
        return render_template('menu_res.html', final2=final2,final=final, result=result, menu=menu,
                                fname=fname, mtime=mtime)

if __name__ == '__main__':
    app.run()



#리절트 > 딕셔너리, vs code갱신
# 코드간략화 , 프롬임폴트, 전처리와 프레딕트의 분리, 텐서인풋프레딕트
#로고제거
# 경우에 따라 결과하면서에서 바로 이미지가 안뜨고 새로고침해야함  > 로딩중화면을띄우기



 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://a893-35-184-168-124.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [23/Aug/2022 02:56:02] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Aug/2022 02:56:03] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [23/Aug/2022 02:56:27] "GET /menu HTTP/1.1" 200 -
100%|██████████| 1/1 [00:03<00:00,  3.59s/it]
INFO:werkzeug:127.0.0.1 - - [23/Aug/2022 02:57:30] "POST /menu HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Aug/2022 02:57:31] "GET /upload/upload/aa.jpg?q=1661223393 HTTP/1.1" 200 -
